# HUAPize Any Agent Workflow

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mircus/HUAP/blob/main/notebooks/HUAPize_Agents.ipynb)

<p align="center">
  <img src="https://raw.githubusercontent.com/Mircus/HUAP/main/HUAP-logo.png" alt="HUAP Logo" width="400"/>
</p>

**Take an existing multi-agent app and make it traceable, diffable, and CI-gatable — in 5 minutes.**

This notebook shows the "before and after" of HUAPizing a plain Python multi-agent workflow
using HUAP's manual tracer. No external framework needed — works with any agent code.

> **Want auto-instrumented LangChain tracing?** See [HUAPize_LangChain.ipynb](https://colab.research.google.com/github/Mircus/HUAP/blob/main/notebooks/HUAPize_LangChain.ipynb)

| Before HUAP | After HUAP |
|---|---|
| Script runs, prints output | Every action recorded as `trace.jsonl` |
| "It worked yesterday" | Deterministic replay + diff |
| No regression testing | CI gates with golden baselines |
| No audit trail | Shareable HTML reports |

In [ ]:
# Install HUAP from GitHub (latest main branch)
!pip install -q "huap-core @ git+https://github.com/Mircus/HUAP.git@main#subdirectory=packages/hu-core"

# Stub mode: deterministic responses, no API keys
import os
os.environ["HUAP_LLM_MODE"] = "stub"

---
## Step 1: A Basic Multi-Agent App (before HUAP)

Here's a typical multi-agent workflow: a **researcher** searches the web and summarizes findings,
then a **writer** drafts a memo. This pattern is common in CrewAI, AutoGen, and plain Python agents.

It works — but there's no trace, no CI, no way to reproduce or diff runs.

In [ ]:
# === BEFORE HUAP: a bare multi-agent workflow ===

def researcher(topic):
    """Agent 1: search + summarize."""
    # Simulate tool call
    search_results = [{"title": f"Survey of {topic}", "source": "arxiv"}]
    # Simulate LLM call
    summary = f"{topic}: deterministic tracing is the top differentiator for agent CI."
    return {"search_results": search_results, "summary": summary}

def writer(research):
    """Agent 2: draft a memo from research."""
    # Simulate LLM call
    memo = f"# Research Memo\n\n{research['summary']}\n\nBased on {len(research['search_results'])} sources."
    return {"memo": memo}

# Run the pipeline
research = researcher("AI agent frameworks")
result = writer(research)

print(result["memo"])
print("\n--- That's it. No trace. No way to diff. No CI. ---")

---
## Step 2: HUAPize It — Add Tracing

Same workflow, now wrapped with HUAP's manual tracer.

Three changes:
1. Import `huap_trace_crewai` context manager
2. Wrap your workflow in `with huap_trace_crewai(...) as tracer:`
3. Call `tracer.on_*()` methods at each step

That's it. Every action is now recorded as a JSONL trace event.

> **Note:** This uses HUAP's manual tracer — you control what gets recorded.
> For frameworks with callback APIs (like LangChain), HUAP can auto-instrument with zero code changes.

In [ ]:
# === AFTER HUAP: same workflow, now traced ===

import json
from pathlib import Path
from hu_core.adapters.crewai import huap_trace_crewai

Path("traces").mkdir(exist_ok=True)

with huap_trace_crewai(out="traces/crewai_demo.jsonl", run_name="crewai_demo") as tracer:

    # --- Agent 1: researcher ---
    tracer.on_agent_step("researcher", "Search for AI agent frameworks")

    tracer.on_tool_call("web_search", {"query": "AI agent frameworks 2025"})
    search_results = [{"title": "Survey of AI agent frameworks", "source": "arxiv"}]
    tracer.on_tool_result("web_search", {"results": search_results}, duration_ms=95)

    tracer.on_llm_request("gpt-4o", [{"role": "user", "content": "Summarize findings on AI agent frameworks"}])
    summary = "AI agent frameworks: deterministic tracing is the top differentiator for agent CI."
    tracer.on_llm_response(
        "gpt-4o", summary,
        usage={"prompt_tokens": 18, "completion_tokens": 14, "total_tokens": 32},
        duration_ms=320,
    )

    # --- Agent 2: writer ---
    tracer.on_agent_step("writer", "Draft a memo from research")

    tracer.on_llm_request("gpt-4o", [{"role": "user", "content": "Write a research memo"}])
    memo = f"# Research Memo\n\n{summary}\n\nBased on {len(search_results)} sources."
    tracer.on_llm_response(
        "gpt-4o", memo,
        usage={"prompt_tokens": 14, "completion_tokens": 20, "total_tokens": 34},
        duration_ms=280,
    )

# --- Show the trace ---
print("=" * 60)
print("Trace events (traces/crewai_demo.jsonl):")
print("=" * 60)
with open("traces/crewai_demo.jsonl") as f:
    for line in f:
        event = json.loads(line)
        kind = event.get("kind", "?")
        name = event.get("name", "?")
        data = event.get("data", {})
        label = data.get("tool", data.get("model", data.get("node", "")))
        print(f"  {kind:12s}  {name:20s}  {label}")

---
## Step 3: Generate a Shareable HTML Report

Turn the raw JSONL into a standalone HTML report anyone can open in a browser.

In [ ]:
from pathlib import Path

!huap trace report traces/crewai_demo.jsonl --out reports/crewai_demo.html 2>&1 || true

report = Path("reports/crewai_demo.html")
if report.exists():
    from IPython.display import HTML, display
    print("=" * 60)
    print("HTML Report (inline):")
    print("=" * 60)
    display(HTML(report.read_text()))
else:
    print("Report generation requires huap trace report command.")
    print("The trace file is still at: traces/crewai_demo.jsonl")

---
## Step 4: Prove Reproducibility — Diff Two Runs

Run the same workflow again and diff the two traces.
In stub mode, they should be identical — proving deterministic replay.

In [ ]:
# Run the same workflow a second time
with huap_trace_crewai(out="traces/crewai_demo_v2.jsonl", run_name="crewai_demo_v2") as tracer:
    tracer.on_agent_step("researcher", "Search for AI agent frameworks")
    tracer.on_tool_call("web_search", {"query": "AI agent frameworks 2025"})
    tracer.on_tool_result("web_search", {"results": [{"title": "Survey of AI agent frameworks", "source": "arxiv"}]}, duration_ms=95)
    tracer.on_llm_request("gpt-4o", [{"role": "user", "content": "Summarize findings on AI agent frameworks"}])
    tracer.on_llm_response("gpt-4o", "AI agent frameworks: deterministic tracing is the top differentiator for agent CI.",
                           usage={"prompt_tokens": 18, "completion_tokens": 14, "total_tokens": 32}, duration_ms=320)
    tracer.on_agent_step("writer", "Draft a memo from research")
    tracer.on_llm_request("gpt-4o", [{"role": "user", "content": "Write a research memo"}])
    tracer.on_llm_response("gpt-4o", "# Research Memo\n\nAI agent frameworks: deterministic tracing is the top differentiator for agent CI.\n\nBased on 1 sources.",
                           usage={"prompt_tokens": 14, "completion_tokens": 20, "total_tokens": 34}, duration_ms=280)

# Diff the two runs
!huap trace diff traces/crewai_demo.jsonl traces/crewai_demo_v2.jsonl

---
## What You Just Did

| Before | After |
|---|---|
| A Python script that runs and prints | A traced, diffable, CI-gatable agent system |
| No audit trail | Every action in `trace.jsonl` |
| "It worked yesterday" | Deterministic replay proves reproducibility |
| No regression testing | `huap ci run` gates regressions against baselines |
| No shareable artifacts | `trace.html` — send to anyone |

### What changed in your code?

Three lines:
1. `from hu_core.adapters.crewai import huap_trace_crewai`
2. `with huap_trace_crewai(out=...) as tracer:`
3. `tracer.on_*()` calls at each agent step

Your existing logic stays exactly the same.

### Next steps
- [GitHub](https://github.com/Mircus/HUAP) — source, examples, contributing guide
- [Try HUAP](https://colab.research.google.com/github/Mircus/HUAP/blob/main/notebooks/Try_HUAP.ipynb) — the flagship demo notebook
- [Getting Started](https://github.com/Mircus/HUAP/blob/main/GETTING_STARTED.md) — full tutorial
- [CONTRIBUTING.md](https://github.com/Mircus/HUAP/blob/main/CONTRIBUTING.md) — join us